# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
!pip install fuzzywuzzy
!pip install openpyxl
!pip install rapidfuzz

In [9]:
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = '/Users/rokokkula/Documents/cprtpr-datastewards-sp1-stewardapp-prbq_2_1.json'
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)

import os
os.environ["GCP_BIGQUERY_KEYFILE"] = "/Users/rokokkula/Documents/cprtpr-datastewards-sp1-stewardapp-prbq_2_1.json"

# Initialize BigQuery client
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
client = bigquery.Client(credentials=credentials, project=your_project_id)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)



In [3]:
query_cmt = """--Lapsed members in the past 12 months for Fuzzy Matching (Stuart, Tim)
-- MBR lapsed paid 12mo current guest members
WITH lapsed_12mo as (select mbr_nbr, event_type, event_value
from `cprtpr-dataplatform-sp1.usviews.v_us_member_events_fact`
where event_type = 'Member Last Lapsed Date'
AND date(event_value) > DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH))
Select m.mbr_nbr, mbr_name, mbr_phone1, mbr_company, mbr_addr, mbr_city, mbr_state, mbr_country
from `cprtpr-dataplatform-sp1`.`usmart.v_us_member_fact` m
join lapsed_12mo l
on m.mbr_nbr = l.mbr_nbr
where mbr_mbrshp_type_cd = 'GUEST'
AND mbr_type IN ('LPSDBASIC', 'LPSDPREMIER')"""

# Execute the query


df = (
    client.query(query_cmt)
          .result()
          .to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type='tqdm_notebook')
)
df.head()

Downloading:   0%|          |

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country
0,342748,Darius Mankowski,BDIlkBokeXUVolnGIw/LUZQa+6t6bgGmCNbFt1o6vXg=,DARIUS MANKOWSKI,124 city road,London,GB,GBR
1,291391,William Hildebrand,epo2sw4PnsmIqlEzOhfOiCYb8lDMJ4aqDKQzi6LOE6M=,WILLIAM HILDEBRAND,12210 Hillcrest dr,Prince George,BC,CAN
2,282494,ABDOULAYE DIANE,None,ABDOULAYE DIANE,7239 Rue de Honfleur,Trois Rivieres,QC,CAN
3,282109,MOISES TORRES AGUILAR,KqcdEWYK+PdZzrRb7mXLXqKXBkQuXhNTdD81fsrNrow=,MOISES TORRES AGUILAR,Cantabrica 15,Puebla,PB,MEX
4,282031,Olexander Kovalenko,None,OLEXANDER KOVALENKO,Mira,Dnipro,UA,UKR


In [15]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=NcauLY29OR5565Eof50zIjyU0reNGy&access_type=offline&code_challenge=p2jn9MdWMWZnqt1C9w07BAQUGR93IravKWqCVgmc6-Q&code_challenge_method=S256


Credentials saved to file: [/Users/rokokkula/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "cprtqa-strategicanalytics-sp1" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some 

In [10]:
query_cmt = """Select mbr_nbr, mbr_name, mbr_phone1, mbr_company, mbr_addr, mbr_city, mbr_state, mbr_country
from `cprtpr-dataplatform-sp1`.`usmart.v_us_member_fact`
where mbr_mbrshp_type_cd = 'GUEST'
   AND date(mbr_first_reg_dt) > DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
and COALESCE(mbr_first_reg_paid_dt,mbr_first_basic_reg_paid_dt) is null
--mbr_nbr is not null,
 and mbr_name is not null
 and mbr_phone1 is not null
 and mbr_company is not null
 and mbr_addr is not null
 and mbr_city is not null
 and mbr_state is not null
 and mbr_country is not null"""


# Execute the query

df = (
    client.query(query_cmt)
          .result()
          .to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type='tqdm_notebook')
)
df.head()

Downloading:   0%|          |

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country
0,310660,Hemat Ghaznavi,5355772099,HEMAT GHAZNAVI,DOOR # 168 NO 68B ST 1988 ESENYURT 34515,ISTANBUL,TR,TUR
1,283944,ANTHONY HILL PEREZ,2105481318,ANTHONY HILL PEREZ,5170 Kamehameha Loop,Honolulu,HI,USA
2,629874,MEGAN LEIGH BARNETT,3043739362,MEGAN LEIGH BARNETT,73 Summer Lane,Given,WV,USA
3,286920,LAIMIS PANKEVICIUS,700558899,LAIMIS PANKEVICIUS,Asenvagen 35,Ucklum,SE,SWE
4,280325,Gena Sakunts,43090619,GENA SAKUNTS,Margaryan 43,Yerevan,AM,ARM


In [7]:
query_cmt = """
Select mbr_nbr, mbr_email,mbr_name, mbr_phone1, mbr_company, mbr_addr, mbr_city, mbr_state, mbr_country
from `cprtpr-dataplatform-sp1`.`usmart.v_us_member_fact`
where mbr_mbrshp_type_cd = 'GUEST'
   AND date(mbr_first_reg_dt) > DATE_SUB(CURRENT_DATE(), INTERVAL 24 MONTH)
and mbr_phone1 is not null or mbr_email is not null"""



df = (
    client.query(query_cmt)
          .result()
          .to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type='tqdm_notebook')
)
df.head()


NameError: name 'client' is not defined

In [5]:
lap=pd.read_excel('data/Copy of SalvageBuyers.xlsx').rename(columns={'Id': 'mbr_nbr'})
lap.head()

,mbr_nbr,mbr_company,mbr_phone1,FirstName,LastName,mbr_name,UserName,mbr_email,mbr_addr,Street2,mbr_city,mbr_state,ZipPostal,LastLoginDate,LastActivityDate
0,1,Great West Casualty Compnay,402-494-2411,Auction,Administrator,Auction Administrator,GW-AuctionAdmin,GW-AuctionAdmin@gwccnet.com,1100 W 29th St,NaN,South Sioux City,NE,68776,2024-06-28 16:19:09,2024-06-28 16:19:09
1,162383,GREAT WEST CASUALTY,402-494-7891,Rich,Vandenberg,Rich Vandenberg,r.vandenberg,r.vandenberg@gwccnet.com,1100 W 29th St,NaN,South Sioux City,NE,68776-3130,2024-12-18 20:20:52,2024-12-18 20:20:52
2,166344,GREAT WEST CASUALTY,817-385-2720,Gary,Rheinlander,Gary Rheinlander,g.rheinlander,g.rheinlander@gwccnet.com,417 Rolling Hills Trl,NaN,Arlington,TX,76011-2247,2021-04-28 12:31:41,2021-04-28 12:31:41
3,166444,great west casualty,402-494-2737,Doug,Brown,Doug Brown,d.brown,d.brown@gwccnet.com,624 Six Flags Dr Ste 240,NaN,Arlington,TX,76011-6370,2024-02-20 15:45:57,2024-02-20 15:45:58
4,167044,GREAT WEST CASUALTY,800-228-8602,Randy,Phillips,Randy Phillips,r.phillips,r.phillips@gwccnet.com,10100 Global Way Ste 100,NaN,Knoxville,TN,37932-1890,2024-12-18 13:51:29,2024-12-18 13:51:29


In [6]:
lap['mbr_phone1'] = lap['mbr_phone1'].str.replace(r'\D', '', regex=True)
lap.head()

,mbr_nbr,mbr_company,mbr_phone1,FirstName,LastName,mbr_name,UserName,mbr_email,mbr_addr,Street2,mbr_city,mbr_state,ZipPostal,LastLoginDate,LastActivityDate
0,1,Great West Casualty Compnay,4024942411,Auction,Administrator,Auction Administrator,GW-AuctionAdmin,GW-AuctionAdmin@gwccnet.com,1100 W 29th St,NaN,South Sioux City,NE,68776,2024-06-28 16:19:09,2024-06-28 16:19:09
1,162383,GREAT WEST CASUALTY,4024947891,Rich,Vandenberg,Rich Vandenberg,r.vandenberg,r.vandenberg@gwccnet.com,1100 W 29th St,NaN,South Sioux City,NE,68776-3130,2024-12-18 20:20:52,2024-12-18 20:20:52
2,166344,GREAT WEST CASUALTY,8173852720,Gary,Rheinlander,Gary Rheinlander,g.rheinlander,g.rheinlander@gwccnet.com,417 Rolling Hills Trl,NaN,Arlington,TX,76011-2247,2021-04-28 12:31:41,2021-04-28 12:31:41
3,166444,great west casualty,4024942737,Doug,Brown,Doug Brown,d.brown,d.brown@gwccnet.com,624 Six Flags Dr Ste 240,NaN,Arlington,TX,76011-6370,2024-02-20 15:45:57,2024-02-20 15:45:58
4,167044,GREAT WEST CASUALTY,8002288602,Randy,Phillips,Randy Phillips,r.phillips,r.phillips@gwccnet.com,10100 Global Way Ste 100,NaN,Knoxville,TN,37932-1890,2024-12-18 13:51:29,2024-12-18 13:51:29


In [ ]:
merge=pd.merge(lap,df,how='inner',on=['mbr_phone1'])
merge.head()




In [ ]:
merge.to_csv("data/merge2.csv")

In [4]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=peIR0eeFdjrEh1WOBJoDSUvZYOMeHJ&access_type=offline&code_challenge=IeqUZVgS0qdMBaa7_0uIW7kde26V0F1568keE0lafiw&code_challenge_method=S256


Credentials saved to file: [/Users/rokokkula/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "cprtqa-strategicanalytics-sp1" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some 

In [4]:
act=pd.read_csv('data/MBR_active_paid_members.csv').dropna()
act.shape

(307570, 8)

In [12]:
import pandas as pd
import numpy as np
from numpy import mean,std
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,Normalizer,MinMaxScaler
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sklearn.metrics as metrics


import plotly.express as px


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [1]:
import pandas as pd
import numpy as np
from numpy import mean,std
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,Normalizer,MinMaxScaler
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sklearn.metrics as metrics


import plotly.express as px


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#act=pd.read_csv('anant_17k.csv').rename(columns={'mber_name': 'mbr_name'})

#act=pd.read_csv('Lapsed paid members Sep 2024.csv')
lap=pd.read_excel('data/Copy of SalvageBuyers.xlsx').rename(columns={'Id': 'mbr_nbr'})
#act=pd.read_csv('data/MBR_active_paid_members.csv').dropna()
act=df.copy().dropna()
#lap=pd.read_excel('New Premiers Nov 2024.xlsx').dropna()
#lap=pd.read_excel('Anant - Truly new members promo code analysis.xlsx')
lap['mbr_country']='USA'
lap['mbr_phone1'] = lap['mbr_phone1'].str.replace(r'^(?:\(\+\d+\))|\D', '', regex=True)
lap['mbr_phone1'] = lap['mbr_phone1'].astype(float)

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

/opt/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


NameError: name 'df' is not defined

In [5]:
lap.head()
lap.tail(5)
act.head(5)
act.tail(5)

,mbr_nbr,mbr_company,mbr_phone1,FirstName,LastName,mbr_name,UserName,mbr_email,mbr_addr,Street2,...,LastLoginDate,LastActivityDate,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,1,GREAT WEST CASUALTY COMPNAY,4.024942e+09,Auction,Administrator,AUCTION ADMINISTRATOR,GW-AuctionAdmin,GW-AuctionAdmin@gwccnet.com,1100 W 29th St,NaN,...,2024-06-28 16:19:09,2024-06-28 16:19:09,USA,GREAT WEST CASUALTY COMPNAY,1100 W 29TH,GREAT WEST CASUALTY COMPNAY 1100 W 29TH,USA NE SOUTH SIOUX CITY 1100 W 29TH,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMPNAY,1
1,162383,GREAT WEST CASUALTY,4.024948e+09,Rich,Vandenberg,RICH VANDENBERG,r.vandenberg,r.vandenberg@gwccnet.com,1100 W 29th St,NaN,...,2024-12-18 20:20:52,2024-12-18 20:20:52,USA,GREAT WEST CASUALTY,1100 W 29TH,GREAT WEST CASUALTY 1100 W 29TH,USA NE SOUTH SIOUX CITY 1100 W 29TH,RICH VANDENBERG GREAT WEST CASUALTY USA NE SOU...,RICH VANDENBERG GREAT WEST CASUALTY,1
2,166344,GREAT WEST CASUALTY,8.173853e+09,Gary,Rheinlander,GARY RHEINLANDER,g.rheinlander,g.rheinlander@gwccnet.com,417 Rolling Hills Trl,NaN,...,2021-04-28 12:31:41,2021-04-28 12:31:41,USA,GREAT WEST CASUALTY,417 ROLLING HILLS TRL,GREAT WEST CASUALTY 417 ROLLING HILLS TRL,USA TX ARLINGTON 417 ROLLING HILLS TRL,GARY RHEINLANDER GREAT WEST CASUALTY USA TX AR...,GARY RHEINLANDER GREAT WEST CASUALTY,1
3,166444,GREAT WEST CASUALTY,4.024943e+09,Doug,Brown,DOUG BROWN,d.brown,d.brown@gwccnet.com,624 Six Flags Dr Ste 240,NaN,...,2024-02-20 15:45:57,2024-02-20 15:45:58,USA,GREAT WEST CASUALTY,624 SIX FLAGS E 240,GREAT WEST CASUALTY 624 SIX FLAGS E 240,USA TX ARLINGTON 624 SIX FLAGS E 240,DOUG BROWN GREAT WEST CASUALTY USA TX ARLINGTO...,DOUG BROWN GREAT WEST CASUALTY,1
4,167044,GREAT WEST CASUALTY,8.002289e+09,Randy,Phillips,RANDY PHILLIPS,r.phillips,r.phillips@gwccnet.com,10100 Global Way Ste 100,NaN,...,2024-12-18 13:51:29,2024-12-18 13:51:29,USA,GREAT WEST CASUALTY,10100 GLOBAL WAY E 100,GREAT WEST CASUALTY 10100 GLOBAL WAY E 100,USA TN KNOXVILLE 10100 GLOBAL WAY E 100,RANDY PHILLIPS GREAT WEST CASUALTY USA TN KNOX...,RANDY PHILLIPS GREAT WEST CASUALTY,1


,mbr_nbr,mbr_company,mbr_phone1,FirstName,LastName,mbr_name,UserName,mbr_email,mbr_addr,Street2,...,LastLoginDate,LastActivityDate,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
1346,47327521,GWCC,4.024947e+09,Stacey,Oerman,STACEY OERMAN,msoerman,msoerman@yahoo.com,PO BOX 94,NaN,...,2024-12-18 21:43:38,2024-12-18 21:43:39,USA,GWCC,PO BOX 94,GWCC PO BOX 94,USA NE SOUTH SIOUX CITY PO BOX 94,STACEY OERMAN GWCC USA NE SOUTH SIOUX CITY PO ...,STACEY OERMAN GWCC,1
1347,47340076,GREAT WEST CASUALTY,NaN,Daniela,Garcia,DANIELA GARCIA,dgarci3,d.garcia3@gwccnet.com,1100 W 29th ST,NaN,...,2024-12-17 14:52:18,2024-12-17 14:52:18,USA,GREAT WEST CASUALTY,1100 W 29TH,GREAT WEST CASUALTY 1100 W 29TH,USA NE SOUTH SIOUX CITY 1100 W 29TH,DANIELA GARCIA GREAT WEST CASUALTY USA NE SOUT...,DANIELA GARCIA GREAT WEST CASUALTY,1
1348,47340117,GREAT WEST CASUALTY CO,8.123374e+09,Roberta,Eller,ROBERTA ELLER,R.Eller,r.eller@gwccnet.com,3042 North Lintel Drive,NaN,...,2024-12-16 14:51:57,2024-12-16 14:56:29,USA,GREAT WEST CASUALTY CO,3042 NORTH LINTEL,GREAT WEST CASUALTY CO 3042 NORTH LINTEL,USA IN BLOOMINGTON 3042 NORTH LINTEL,ROBERTA ELLER GREAT WEST CASUALTY CO USA IN BL...,ROBERTA ELLER GREAT WEST CASUALTY CO,1
1349,47347325,GREAT WEST CASUALTY COMPANY,NaN,Brenda,Cohen,BRENDA COHEN,BSCPDC,B.COHEN@GWCCNET.COM,3042 N Lintel Dr,NaN,...,2024-12-18 16:17:30,2024-12-18 16:17:30,USA,GREAT WEST CASUALTY COMPANY,3042 N LINTEL,GREAT WEST CASUALTY COMPANY 3042 N LINTEL,USA IN BLOOOMINGTON 3042 N LINTEL,BRENDA COHEN GREAT WEST CASUALTY COMPANY USA I...,BRENDA COHEN GREAT WEST CASUALTY COMPANY,1
1350,47347876,GREAT WEST CASUALTY COMPANY,8.123374e+09,Jackie,Branaman,JACKIE BRANAMAN,j.branaman@gwccnet.com,j.branaman@gwccnet.com,3042 N. LINTEL DRIVE,NaN,...,2024-12-18 16:09:00,2024-12-18 16:09:00,USA,GREAT WEST CASUALTY COMPANY,3042 N. LINTEL,GREAT WEST CASUALTY COMPANY 3042 N. LINTEL,USA IN BLOOMINGTON 3042 N. LINTEL,JACKIE BRANAMAN GREAT WEST CASUALTY COMPANY US...,JACKIE BRANAMAN GREAT WEST CASUALTY COMPANY,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,342748,DARIUS MANKOWSKI,BDIlkBokeXUVolnGIw/LUZQa+6t6bgGmCNbFt1o6vXg=,DARIUS MANKOWSKI,124 city road,LONDON,GB,GBR,DARIUS MANKOWSKI,124 CITY,DARIUS MANKOWSKI 124 CITY,GBR GB LONDON 124 CITY,DARIUS MANKOWSKI DARIUS MANKOWSKI GBR GB LONDO...,DARIUS MANKOWSKI DARIUS MANKOWSKI,0
1,291391,WILLIAM HILDEBRAND,epo2sw4PnsmIqlEzOhfOiCYb8lDMJ4aqDKQzi6LOE6M=,WILLIAM HILDEBRAND,12210 Hillcrest dr,PRINCE GEORGE,BC,CAN,WILLIAM HILDEBRAND,12210 HILLCRE,WILLIAM HILDEBRAND 12210 HILLCRE,CAN BC PRINCE GEORGE 12210 HILLCRE,WILLIAM HILDEBRAND WILLIAM HILDEBRAND CAN BC P...,WILLIAM HILDEBRAND WILLIAM HILDEBRAND,0
3,282109,MOISES TORRES AGUILAR,KqcdEWYK+PdZzrRb7mXLXqKXBkQuXhNTdD81fsrNrow=,MOISES TORRES AGUILAR,Cantabrica 15,PUEBLA,PB,MEX,MOISES TORRES AGUILAR,CANTABRICA 15,MOISES TORRES AGUILAR CANTABRICA 15,MEX PB PUEBLA CANTABRICA 15,MOISES TORRES AGUILAR MOISES TORRES AGUILAR ME...,MOISES TORRES AGUILAR MOISES TORRES AGUILAR,0
5,283068,HARRY ALBROW,rIoV20V1M/it/ONUkybTzyVpOu3oY41WrBwYWIOHGbE=,HARRY ALBROW,Farnerhaldenstrasse 2,ZUERICH,SZ,SWZ,HARRY ALBROW,FARNERHALDENRASSE 2,HARRY ALBROW FARNERHALDENRASSE 2,SWZ SZ ZUERICH FARNERHALDENRASSE 2,HARRY ALBROW HARRY ALBROW SWZ SZ ZUERICH FARNE...,HARRY ALBROW HARRY ALBROW,0
6,282289,SAMI ALI,IpdGiutToLVmOagIeovsvUCaIuEcOBSOPdUX5UlNWJs=,SAMI ALI,Gurji st,TRIPOLI,LY,LBY,SAMI ALI,GURJI,SAMI ALI GURJI,LBY LY TRIPOLI GURJI,SAMI ALI SAMI ALI LBY LY TRIPOLI GURJI,SAMI ALI SAMI ALI,0


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
201560,669081,HERMELINDO MOTA,Ow256CJqjeYKwTi8BfaPUvUQFpzDhZ+xPoZMqvrf8+4=,HERMELINDO MOTA,1800 W Aspen st,MILWAUKEE,WI,USA,HERMELINDO MOTA,1800 W ASPEN,HERMELINDO MOTA 1800 W ASPEN,USA WI MILWAUKEE 1800 W ASPEN,HERMELINDO MOTA HERMELINDO MOTA USA WI MILWAUK...,HERMELINDO MOTA HERMELINDO MOTA,0
201561,767395,S T CROSS JR,6NZK5A6Fm7CmRcqhlacwHWJeoYUGvGH+cKKM/Rzd4Ws=,S T CROSS JR,3630 N 14th st,MILWAUKEE,WI,USA,S T CROSS JR,3630 N 14TH,S T CROSS JR 3630 N 14TH,USA WI MILWAUKEE 3630 N 14TH,S T CROSS JR S T CROSS JR USA WI MILWAUKEE 363...,S T CROSS JR S T CROSS JR,0
201562,670034,COREY CAPELLI,XbatFfBZsPXs25oR3F0flad02JM7hAIv8IR5hRJnOyo=,COREY CAPELLI,2718 83rd Place Kenosha,KENOSHA,WI,USA,COREY CAPELLI,2718 83 PLACE KENOSHA,COREY CAPELLI 2718 83 PLACE KENOSHA,USA WI KENOSHA 2718 83 PLACE KENOSHA,COREY CAPELLI COREY CAPELLI USA WI KENOSHA 271...,COREY CAPELLI COREY CAPELLI,0
201563,140703,ALAEDDIN NOURI,268JxSUKDW3VOynXNuirRV1V8QLbpmJ4vFX2XCsKUCw=,ALAEDDIN NOURI,4295 s. Whitnall Ave unit r,MILWAUKEE,WI,USA,ALAEDDIN NOURI,4295 S. WHITNALL R,ALAEDDIN NOURI 4295 S. WHITNALL R,USA WI MILWAUKEE 4295 S. WHITNALL R,ALAEDDIN NOURI ALAEDDIN NOURI USA WI MILWAUKEE...,ALAEDDIN NOURI ALAEDDIN NOURI,0
201564,363855,DENNIS GARVEN,HrgPofjR7FD7c8yctssR7lGeEIgf+85ORqlH92Sxb/0=,DENNIS GARVEN,4740 South 99th street,GREENFIELD,WI,USA,DENNIS GARVEN,4740 SOUTH 99TH,DENNIS GARVEN 4740 SOUTH 99TH,USA WI GREENFIELD 4740 SOUTH 99TH,DENNIS GARVEN DENNIS GARVEN USA WI GREENFIELD ...,DENNIS GARVEN DENNIS GARVEN,0


In [6]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('Dave Request lapsed 2.csv')

--- 1778.627610206604 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,634933,DAVID GARCIA BAUTISTA,64gnEeMQeH1T1VomQH2Nw+uQRTNdWrBbtZdzL2EZ8ys=,GREAT MOTORS AUTO SALES LLC,6356 VAN NUYS BLVD STE228,DAVID GARCIA BAUTISTA GREAT USA CA VAN NUY...,0,42.028986,47.368421,40.000000,42.857143,100.000000,hier 4
1,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,705490,ALI ALHAYKI,qCFB8KwgNk+sXqpR7AzXRFW0b/ysAwZ2d4oy14/YvD8=,GREAT AUTOS,310 3RD AVE,ALI ALHAYKI GREAT S USA IN TERRE HAUTE 310 3,0,48.062016,41.666667,47.457627,31.250000,83.333333,hier 4
2,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,68857,LUIS MAGANA,9SwsFZwQQSZe4+Reqf+gNBYgi3CJhbRNNVxzVh22AdM=,LUIS MAGANA,403 w 17 th st,LUIS MAGANA LUIS MAGANA USA NE SOUTH SIOUX CIT...,0,69.444444,26.666667,88.571429,31.250000,31.578947,hier 4
3,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,662591,FABIAN HERNANDEZ,YeiHSO6EQCZBesRvb2yyZrSLySmb+mmGJZNBe11MUDc=,FABIAN HERNANDEZ,2313 A st,FABIAN HERNANDEZ FABIAN HERNANDEZ USA NE SOUTH...,0,65.714286,24.615385,86.792453,32.432432,32.558140,hier 4
4,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,62363,GRAHAM PALSMA,b0jHLFB5nKkWEyO6DeRIJ+h6F4M75Zfiv+qFekyM8Gg=,GRAHAM PALSMA,2904 G St,GRAHAM PALSMA GRAHAM PALSMA USA NE SOUTH SIOUX...,0,68.656716,25.806452,86.792453,23.529412,30.000000,hier 4


In [16]:
import os
os.getcwd()

'/Users/rokokkula/Documents/scientificProject'

In [29]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
#actx = act.copy()
actx = lap.copy()

start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80))and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('Dave Request buyer analysis.csv')

--- 14.991272926330566 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,162383,RICH VANDENBERG,4.024948e+09,GREAT WEST CASUALTY,1100 W 29th St,RICH VANDENBERG GREAT WEST CASUALTY USA NE SOU...,1,87.301587,70.370370,100.000000,22.222222,100.0,hier 3
1,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,166344,GARY RHEINLANDER,8.173853e+09,GREAT WEST CASUALTY,417 Rolling Hills Trl,GARY RHEINLANDER GREAT WEST CASUALTY USA TX AR...,1,56.250000,69.090909,41.095890,27.027027,100.0,hier 4
2,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,166444,DOUG BROWN,4.024943e+09,GREAT WEST CASUALTY,624 Six Flags Dr Ste 240,DOUG BROWN GREAT WEST CASUALTY USA TX ARLINGTO...,1,60.526316,77.551020,50.704225,25.806452,100.0,hier 4
3,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,167044,RANDY PHILLIPS,8.002289e+09,GREAT WEST CASUALTY,10100 Global Way Ste 100,RANDY PHILLIPS GREAT WEST CASUALTY USA TN KNOX...,1,54.088050,71.698113,48.648649,34.285714,100.0,hier 4
4,1,AUCTION ADMINISTRATOR,4.024942e+09,GREAT WEST CASUALTY COMPNAY,1100 W 29th St,AUCTION ADMINISTRATOR GREAT WEST CASUALTY COMP...,1,182174,JOHN ZEVENBERGEN,4.024947e+09,GREAT WEST CASUALTY,1100 W 29th St,JOHN ZEVENBERGEN GREAT WEST CASUALTY USA NE SO...,1,86.614173,69.090909,100.000000,16.216216,100.0,hier 3


In [24]:
lap.shape[0]

1351

In [12]:
with ThreadPoolExecutor(max_workers=64) as executor:
        final_results = list(executor.map(process_row, list(lap['mbr_nbr'].head(100))))
        final_df = pd.concat(final_results, ignore_index=True)

TypeError: Cannot index by location index with a non-integer key

In [7]:
from joblib import Parallel, delayed
import time
from concurrent.futures import ThreadPoolExecutor
import os
from tqdm import tqdm

In [13]:
with ThreadPoolExecutor(max_workers=64) as executor:
        final_results = list(executor.map(process_row, list(lap['mbr_nbr'].head(100))))
        final_df = pd.concat(final_results, ignore_index=True)
 
 

TypeError: Cannot index by location index with a non-integer key

In [27]:
lap['mbr_company'].value_counts().head(20)

mbr_company
GREAT WEST CASUALTY COMPANY         44
GREAT WEST CASUALTY                 37
GWCC                                16
GREAT WEST                           7
GREAT WEST CASUALTY CO.              6
GREAT WEST CASUALTY CO               4
DAKOTA COUNTRY TRUCKS                3
ROCKY MOUNTAIN TRUCK PARTS           3
BAYS TRUCK SALVAGE                   2
MID-STATES UTILITY TRAILER SALES     2
TRANSPORT REFRIGERATION INC          2
UNIVERSAL TRUCK SALES                2
TRUCK PARTS & EQUIPMENT              2
LKQ                                  2
BARCLAY TRUCK REBUILDERS             2
DIOR MOTORS LLC                      2
WELLER SALVAGE                       2
SPALDING AUTO PARTS                  2
THERMO KING CHRISTENSEN              2
GREAT WEST CASULATY                  2
Name: count, dtype: int64

In [13]:
x=pd.read_excel("data/Book2.xlsx")
x.head()


,output_details_json
0,"{""fr"": {""image_url"": ""https://c-static.copart...."
1,"{""fr"": {""image_url"": ""https://c-static.copart...."
2,"{""fr"": {""image_url"": ""https://c-static.copart...."
3,"{""fr"": {""image_url"": ""https://c-static.copart...."
4,"{""fr"": {""image_url"": ""https://c-static.copart...."


In [14]:
x.head()

,output_details_json
0,"{""fr"": {""image_url"": ""https://c-static.copart...."
1,"{""fr"": {""image_url"": ""https://c-static.copart...."
2,"{""fr"": {""image_url"": ""https://c-static.copart...."
3,"{""fr"": {""image_url"": ""https://c-static.copart...."
4,"{""fr"": {""image_url"": ""https://c-static.copart...."


In [18]:
import pandas as pd
import json

# Sample DataFrame
data = {
    'output_details_json': [
        '{"fr": {"image_url": "https://c-static.copart.com/v1/AUTH_svc.pdoc00001/PIX1234.jpg"}, "fl": {"image_url": "https://c-static.copart.com/v1/AUTH_svc.pdoc00001/PIX5678.jpg"}}',
        '{"rr": {"image_url": "https://c-static.copart.com/v1/AUTH_svc.pdoc00001/PIX9101.jpg"}, "rl": {"image_url": "https://c-static.copart.com/v1/AUTH_svc.pdoc00001/PIX1121.jpg"}}'
    ]
}
df = pd.DataFrame(data)

# Function to extract image URLs
def extract_image_urls(json_str):
    try:
        data = json.loads(json_str)
        urls = {
            'fr': data.get('fr', {}).get('image_url', None),
            'fl': data.get('fl', {}).get('image_url', None),
            'rr': data.get('rr', {}).get('image_url', None),
            'rl': data.get('rl', {}).get('image_url', None)
        }
        return urls
    except json.JSONDecodeError:
        return {'fr': None, 'fl': None, 'rr': None, 'rl': None}

# Apply the function to the DataFrame column
x['image_urls'] = x['output_details_json'].apply(extract_image_urls)

# Expand the dictionary into separate columns
df = x.join(pd.json_normalize(x['image_urls']))

# Drop the intermediate columns
df = df.drop(columns=['output_details_json', 'image_urls'])

print(df)

                                                    fr  \
0    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
1    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
2    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
3    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
4    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
..                                                 ...   
995  https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
996  https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
997  https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
998  https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
999  https://c-static.copart.com/v1/AUTH_svc.pdoc00...   

                                                    fl  \
0    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
1    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
2    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
3    https://c-static.copart.com/v1/AUTH_svc.pdoc00...   
4    https://

In [19]:
df.head()

,fr,fl,rr,rl
0,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
1,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
2,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
3,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
4,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...


In [20]:
df.to_csv('imageurls.csv')

In [21]:



4 images 
each image has 6 panel 

for i in image:
    for j in panel_i score:
        if (j>0.1)
            ct_dmg_panel[i]=+1
 
for i in image:
    if (ct_dmg_panel[i]>0):
        ct_dmg_image=+1
        sum=+ct_dmg_panel

if ct_dmg_img<=2 & sum<=4
        light =YES

SyntaxError: invalid syntax (2193693383.py, line 1)